In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [2]:
r = requests.get('https://technave.com/gadgets/Gadget-Review-Malaysia-72.html')

In [4]:
soup = BeautifulSoup(r.content, "lxml")

In [20]:
menu = soup.find('div', {'class': 'menu'}).find_all('a')
menu = [a_.get('href') for a_ in menu]
menu = [a_ for a_ in menu if 'http' in a_ and 'html' in a_]
menu

['https://technave.com/gadgets/Gadget-News-Malaysia-49.html',
 'https://technave.com/gadgets/Gadget-Review-Malaysia-72.html',
 'https://technave.com/gadgets/Mobile-Phone-Price-Malaysia-10.html',
 'https://technave.com/gadgets/Laptop-Price-Malaysia-119.html',
 'https://technave.com/gadgets/iPad-Tablet-Price-Malaysia-37.html',
 'https://technave.com/gadgets/Smartwatch-Price-Malaysia-92.html',
 'https://technave.com/gadgets/Camera-Price-Malaysia-26.html',
 'https://technave.com/comparison/compare-phone-specs-price-malaysia-10.html',
 'https://technave.com/comparison/compare-phone-specs-price-malaysia-10.html',
 'https://technave.com/comparison/compare-laptop-specs-price-malaysia-119.html',
 'https://technave.com/comparison/compare-tablet-specs-price-malaysia-37.html',
 'https://technave.com/comparison/compare-smartwatch-specs-price-malaysia-92.html']

In [27]:
menu[0].split('/')[-1].replace('.html', '')

'Gadget-News-Malaysia-49'

In [21]:
r = requests.get(menu[0])
soup = BeautifulSoup(r.content, "lxml")
try:
    max_page = int(soup.find('a', {'class': 'nonce'}).text.split('/')[1].strip())
except:
    max_page = 1

articles = soup.find_all('li', {'class': 'articlelist'})
articles = [a.find('a').get('href') for a in articles]

In [32]:
def crawl(url, filename):
    if os.path.exists(filename):
        return True
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    articles = soup.find_all('li', {'class': 'articlelist'})
    articles = [a.find('a').get('href') for a in articles]
    
    with open(filename, 'w') as fopen:
        json.dump(articles, fopen)

In [33]:
!rm -rf url
!mkdir url

In [35]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for menu_ in menu:
    r = requests.get(menu_)
    t = menu_.split('/')[-1].replace('.html', '')
    print(t)
    soup = BeautifulSoup(r.content, "lxml")
    try:
        max_page = int(soup.find('a', {'class': 'nonce'}).text.split('/')[1].strip())
    except:
        max_page = 1
        
    for i in tqdm(range(0, max_page, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [(f'{menu_}?page={a}', 
                 os.path.join('url', f'{t}-{a}.json')) for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url[0], url[1]): url for url in urls}
        
        
        for future in as_completed(futures):
            if not future.result():
                done = True
        


Gadget-News-Malaysia-49


100%|█████████████████████████████████████████| 233/233 [19:18<00:00,  4.97s/it]


Gadget-Review-Malaysia-72


100%|███████████████████████████████████████████| 19/19 [01:39<00:00,  5.22s/it]


Mobile-Phone-Price-Malaysia-10


...

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.01it/s]


compare-tablet-specs-price-malaysia-37


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.05it/s]


compare-smartwatch-specs-price-malaysia-92


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.05it/s]
